# 1. Install and Import Dependencies

In [77]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler, LabelEncoder
le = LabelEncoder()
# std = StandardScaler()

# 2. Load the data

In [78]:
df = pd.read_csv("datasets/Crop_recommendation.csv") 

# 3. Pre-processing the data

In [99]:
x = df.drop(columns="label")
y = pd.DataFrame(df['label'],columns=['label'])
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0, train_size = .75)
print(y_train)
y_train["label"] = le.fit_transform(y_train["label"])
# 
y_test["label"] = le.transform(y_test["label"])
# y_test

          label
1975     cotton
1287     grapes
760   blackgram
267    chickpea
2007       jute
...         ...
1033     banana
1731     papaya
763   blackgram
835      lentil
1653     orange

[1650 rows x 1 columns]


In [80]:
x_train

,N,P,K,temperature,humidity,ph,rainfall
1975,134,56,18,23.808346,83.919026,6.691268,70.973583
1287,29,122,196,41.948657,81.155952,5.638328,73.068630
760,25,68,19,29.399827,64.255107,7.108450,67.476773
267,41,69,82,20.023815,16.632945,6.715587,68.978065
2007,70,43,40,24.355641,88.803910,6.176860,169.116803
...,...,...,...,...,...,...,...
1033,102,71,48,28.654563,79.286937,5.695268,102.463378
1731,34,68,51,27.347349,94.177567,6.687088,40.351531
763,35,64,15,28.474423,63.536045,6.500145,69.527441
835,39,65,23,25.434598,69.126134,7.685959,41.026829


In [81]:
y_train

,label
1975,6
1287,7
760,2
267,3
2007,8
...,...
1033,1
1731,17
763,2
835,10


In [82]:
y_train = to_categorical(y_train, 22)
y_test = to_categorical(y_test, 22)
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# 4. Model

In [83]:
def crop_predection__model():

    # Define the model
    model = tf.keras.models.Sequential([ 
      tf.keras.layers.Dense(32 , activation='relu'),
      tf.keras.layers.Dense(64 , activation='relu'),
      tf.keras.layers.Dense(22 , activation='softmax'),
  
    ]) 


    # Compile the model
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy']) 
        
    return model

In [84]:
model =crop_predection__model()

In [85]:
history = model.fit(x_train, y_train, epochs=5)

Epoch 1/5
52/52 [==============================] - 0s 1ms/step - loss: 6.8436 - accuracy: 0.1770
Epoch 2/5
52/52 [==============================] - 0s 1ms/step - loss: 1.1439 - accuracy: 0.6497
Epoch 3/5
52/52 [==============================] - 0s 1ms/step - loss: 0.5295 - accuracy: 0.8521
Epoch 4/5
52/52 [==============================] - 0s 1ms/step - loss: 0.3886 - accuracy: 0.8836
Epoch 5/5
52/52 [==============================] - 0s 1ms/step - loss: 0.3156 - accuracy: 0.9055


# 5. Testing

In [86]:
model.evaluate(x_test, y_test, batch_size=50)  

11/11 [==============================] - 0s 1ms/step - loss: 0.2898 - accuracy: 0.9127


[0.28981176018714905, 0.9127272963523865]

In [87]:
model.predict(x_train.iloc[[0]])

1/1 [==============================] - 0s 47ms/step


array([[2.9823883e-07, 9.0837959e-05, 4.3715946e-07, 8.1410371e-11,
        6.6519551e-10, 3.8614949e-06, 8.3601534e-01, 9.5952011e-17,
        7.7284108e-09, 1.7007167e-10, 1.8345193e-07, 1.6376232e-01,
        2.5392030e-14, 5.5632681e-06, 5.0145150e-07, 2.0932777e-05,
        2.5075112e-06, 1.5720370e-05, 4.3772010e-07, 1.3910524e-07,
        7.2840933e-10, 8.1068334e-05]], dtype=float32)

In [97]:
list(le.inverse_transform(y_train))

ValueError: y should be a 1d array, got an array of shape (1650, 22) instead.

In [88]:
def get_label_map(df:pd.DataFrame, label:str):
    """get the mapping between original label and its encoded value
    df: a pandas dataframe with both feature variables and target variable
    label: the name of target variable
    Example:
      df0 = pd.DataFrame({'fea1':[1,2,3,4], 'fea2':['a','b','b','c'], 'target':['cat', 'cat','dog','cat']})
      label = 'target'
      label_map = get_label_map(df=df0, label='target')
    """
    label_map = dict(zip(df["label"], y_le)) # get the mapping between the original labels and encoded labels
    return label_map

In [89]:
label_map = get_label_map(df=y_train, label='label')

C:\Users\Expert Info\AppData\Local\Temp\ipykernel_19928\436202825.py:10: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  y_le = le.fit_transform(df[[label]]) # encode target variable


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices